# Transfer-learning tutorial using DenseNet-121 pre-trained model: example on MedNIST dataset


## Goal of this tutoriel

This tutorial shows how to do 2d images classification example on MedNIST dataset using pretrained PyTorch model.

The goal of this tutorial is to provide an example of transfer learning methods with Fed-BioMed for medical images classification.

### About the model

The model used is Densenet-121 model(“Densely Connected Convolutional Networks”) pretrained on ImageNet dataset. The Pytorch pretrained model [Densenet121](https://pytorch.org/vision/main/models/generated/torchvision.models.html). to perform image classification on the MedNIST dataset. 
The goal of this Densenet121 model is to predict the class of `MedNIST` medical images.



### About MedNIST

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

MedNIST dataset is downloaded from the resources provided by the project [MONAI](https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz)

The dataset MedNIST has 58954 images of size (3, 64, 64) distributed into 6 classes (10000 images per class except for BreastMRI class which has 8954 images). Classes are AbdomenCT, BreastMRI, CXR, ChestCT, Hand, HeadCT. It has the structure:

└── MedNIST/

    ├── AbdomenCT/

    └── BreastMRI/

    └── CXR/

    └── ChestCT/

    └── Hand/

    └── HeadCT/   
   

## Transfer-learning
Transfer learning is a machine learning technique where a model trained on one task is repurposed or adapted for a second related task. Transfer learning uses a pre-trained neural network on a large dataset, as [Imagenet](https://www.image-net.org) is used to train DenseNet model to perform classification of a wide diversity of images.

The objective is that the knowledge gained from learning one task can be useful for learning another task (as we do here, the knowledge of DenseNet model trained on ImageNet is used to classify medical images in 6 categories). This is particularly beneficial when the amount of labeled data for the target task is limited, as the pre-trained model has already learned useful features and representations from a large dataset.

Transfer learning is typically applied in one of two ways:

- (I) Feature Extraction: In this approach, the pre-trained model is used as a fixed feature extractor. The earlier layers of the neural network, which capture general features and patterns, are frozen, and only the later layers are replaced or retrained for the new task. 

- (II) Fine-tuning: In this approach, the pre-trained model is further trained or partially trained on the new task. This allows the model to adapt its learned representations to the specifics of the new task while retaining some of the knowledge gained from the original task.


In this example, we load on the node a sampled dataset ( 500 or 1500 images) of MedNIST to illustrate the effectiveness of the transfer-learning. The sampled dataset is made with a random selection of images and return a sampled dataset with balanced classes, to avoid classification's bias.
We will test these two approches through two independant TrainingPlan experiments. 
To illustrate the effectiveness of these two methods, we load 500 images for the first experiment and 1500 images for the second. The more data you have, the more layers's you can unfreeze for a transfer learning task. 

### 1. Load dataset or sampled dataset
- From the root directory of Fed-BioMed, run :  `source ./scripts/fedbiomed_environment node` in order to load the Node environment
- If you have already ran Mednist nodes before, clean remaining MedNIST nodes : run `./scripts/fedbiomed_run node delete` or `source ./scripts/fedbiomed_environment clean`
- In this new environment, run the script python: `python ./notebooks/transfer-learning/download_sample_of_mednist.py -n <number-of-nodes>`, with `<number-of-nodes>` the number of Nodes you want to create( for more details about this script, please run `notebooks/transfer-learning/download_sample_of_mednist.py --help`)
- The script will ask for each Nodes created the number of samples you want for your dataset. Scripts will output configuration files for each of Nodes, with configured database, using the following naming convention: `config_mednist_<i>_sampled.ini` where `<i>` is ranged from 1 to `<number-of-nodes>` entered.  
- Finally launch your Nodes (one by terminal) by running: `./scripts/fedbiomed_run node config  start config_mednist_<i>_sampled.ini start`, where `<i>` corresponds to the number of Node created.  Wait until you get Starting task manager.

For example, if one wants to create 2 nodes, (`<i>` is equal to 2), one has to run : `python ./notebooks/transfer-learning/download_sample_of_mednist.py -n 2`. One will then launch in seperated terminal `./scripts/fedbiomed_run node config config_mednist_1_sampled.ini start` and `./scripts/fedbiomed_run node config config_mednist_2_sampled.ini start`. Script will ask how many sample should contain the dataset (enter 500 and then 1000).



### 2. Launch the researcher 
- From the root directory of Fed-BioMed, run : `./scripts/fedbiomed_run researcher start`
- It opens the Jupyter notebook.

To make sure that MedNIST dataset is loaded in the node we can send a request to the network to list the available dataset in the node. The list command should output an entry for mednist data.

 

In [1]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

2024-02-23 11:43:12,729 fedbiomed INFO - Starting researcher service...

2024-02-23 11:43:12,747 fedbiomed INFO - Waiting 3s for nodes to connect...

2024-02-23 11:43:13,358 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-23 11:43:13,360 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 11:43:15,760 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 11:43:15,762 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

{'config_mednist_2_sampled': [{'name': 'MedNIST_2_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [1000, 3, 64, 64],
   'dataset_id': 'dataset_6f770f6b-3f0e-4aac-8cc9-60f43880dff5',
   'dataset_parameters': None}],
 'config_mednist_1_sampled': [{'name': 'MedNIST_1_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [500, 3, 64, 64],
   'dataset_id': 'dataset_a6a459ef-91b9-45d0-8ba9-6ce940554dc8',
   'dataset_parameters': None}]}

## Import of librairies 

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan

from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage


## Run an expriment for image's classification using Transfer-learning 

### I- Adapt the last layer to your classification's goal
Here we use the DenseNet model that allows classification through 10000 samples. 
We could adapt this classification's task to the MedNIST dataset by replacing the last layer with our classifier. 
The `model.classifier` layer of the `DenseNet-121` model classifies images through 6 classes, in the Training Plan, by adapting the num_classes value (can be done in through `model_args` argument). 

### Data augmentation
You could perform data augmentation through the preprocess part if you need. Here I show random flip, rotation and crops. 
You could do the preprocessing of images by doing only transforms.resize, transforms.to_tensor and transforms.normalize, as mentionned in the code below (commented lines). 

### I -1. Define Training plan experiment 

In [3]:
class MyTrainingPlan1(TorchTrainingPlan):

    def init_model(self, model_args):
       
        # Load the pre-trained DenseNet model, you have two ways to import your model
        
        model = models.densenet121()
        
        
        # Remove the classification layer of DenseNet
        for param in model.features[:-1].parameters():
            param.requires_grad = False
            
        # add the classifier 
        num_classes = model_args['num_classes'] 
        num_ftrs = model.classifier.in_features
        model.classifier= nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
      
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim",
            "from torchvision.models import densenet121"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    
    # training data
    
    def training_data(self):
        

        # Transform images and  do data augmentation 
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                #transforms.RandomHorizontalFlip(p=0.5),
                #transforms.RandomVerticalFlip(p=0.5),
                #transforms.RandomRotation(30),
                #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
    
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




### Downloading the pretrained model's weights 
Here I download and save the model's weights through Torch.hub using the command below in a file 'pretrained_model.pt'

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121')
torch.save(model.state_dict(), 'pretrained_model.pt')

Using cache found in /home/ybouilla/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-3}, 
    'epochs': 2, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

model_args = {
    'num_classes': 6 # adapt this number to the number of classes in your dataset
}

In [6]:
tags =  ['#MEDNIST', '#dataset']

rounds = 2 # adjsut the number of rounds 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan1,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

# testing section 
from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1) 
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)

2024-02-23 11:43:15,921 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 11:43:15,922 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-23 11:43:15,923 fedbiomed INFO - Node selected for training -> config_mednist_2_sampled

2024-02-23 11:43:15,923 fedbiomed INFO - Node selected for training -> config_mednist_1_sampled

2024-02-23 11:43:15,924 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/Documents/github/fedbiomed/var/experiments/Experiment_0003/model_db321276-8e3c-44d0-a3e8-ead1348765dd.py

...

2024-02-23 11:43:15,999 fedbiomed INFO - Removing tensorboard logs from previous experiment

2024-02-23 11:43:15,999 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-23 11:43:16,000 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-23 11:43:16,000 fedbiomed DEBUG - Experimentation training_args updated for `job`

True

### I - 2. Define the dataset for your experiment 

I propose to run this first experiment with only one Node (ie with  MedNIST_sampled_1 dataset, a sub-sampled dataset of 500 MedNIST images), because this first method is a transfer learning without training.

Here I show how to select one dataset among the connected datasets:

In [7]:

exp.set_nodes(['config_mednist_1_sampled'])
exp.set_tags(['#MEDNIST', '#dataset'])
td = exp.training_data().data()
td.pop('config_mednist_2_sampled')
exp.set_training_data(td)

print(exp.training_data().data())

2024-02-23 11:43:16,004 fedbiomed DEBUG - Experimentation nodes filter changed, you may need to update `training_data`

2024-02-23 11:43:16,005 fedbiomed DEBUG - Experimentation tags changed, you may need to update `training_data`

2024-02-23 11:43:16,005 fedbiomed DEBUG - Training data changed, you may need to update `node_selection_strategy`

2024-02-23 11:43:16,005 fedbiomed DEBUG - Training data changed, you may need to update `job`

2024-02-23 11:43:16,006 fedbiomed DEBUG - Training data changed, you may need to update `aggregator`

{'config_mednist_1_sampled': {'name': 'MedNIST_1_sampled', 'data_type': 'mednist', 'tags': ['#MEDNIST', '#dataset'], 'description': 'MedNIST dataset for transfer learning', 'shape': [500, 3, 64, 64], 'dataset_id': 'dataset_a6a459ef-91b9-45d0-8ba9-6ce940554dc8', 'dtypes': [], 'dataset_parameters': None}}


In [8]:
# importing the downloaded model

exp.training_plan().import_model('pretrained_model.pt')

2024-02-23 11:43:16,043 fedbiomed WARNING - 'TorchModel.set_weights' received inputs that did not cover alltrainable model parameters; missing weights: {'classifier.0.weight', 'classifier.3.weight', 'classifier.3.bias', 'classifier.0.bias'}

2024-02-23 11:43:16,044 fedbiomed WARNING - 'TorchModel.set_weights' received inputs with unexpected names: ['classifier.weight', 'classifier.bias']

### I - 3. Run your experiment 

In [9]:
exp.run()

2024-02-23 11:44:56,527 fedbiomed INFO - Sampled nodes in round 0 ['config_mednist_1_sampled']

2024-02-23 11:44:56,536 fedbiomed INFO - Sending request 
					 To: config_mednist_1_sampled 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-02-23 11:44:56,682 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 11:44:57,942 fedbiomed INFO - TRAINING 
					 NODE_ID: config_mednist_1_sampled 
					 Round 1 Epoch: 1 | Iteration: 1/15 (7%) | Samples: 32/480
 					 Loss: 1.828450 
					 ---------

2024-02-23 11:45:08,504 fedbiomed INFO - TRAINING 
					 NODE_ID: config_mednist_1_sampled 
					 Round 1 Epoch: 1 | Iteration: 10/15 (67%) | Samples: 320/480
 					 Loss: 0.970067 
					 ---------

...

2024-02-23 11:46:02,478 fedbiomed INFO - Nodes that successfully reply in round 1 ['config_mednist_1_sampled']

2024-02-23 11:46:02,543 fedbiomed INFO - Saved aggregated params for round 1 in /home/ybouilla/Documents/github/fedbiomed/var/experiments/Experiment_0003/aggregated_params_b29b2723-7a22-4098-a9a9-f51decb887fd.mpk

2

2024-02-23 12:37:43,481 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 12:42:49,764 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

###### For example,  At the end of training experiment, I obtained

                      INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_41cd99c8-3571-4ab3-958e-6357ce31e91b 
					 Round 2 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 0.980000
					 -

As you can see, Accuracy has been increased in comparison to the first `Expermient`   

### I - 4. Save your model 
You could save your model to later use it in a new TrainingPlan 
This save allows to import the model including your layers's modification and weights values.

In [11]:
#save model 
exp.training_plan().export_model('./training_plan1_densenet_MedNIST')

### I - 5. Results in tensorboard 

In [12]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [13]:
%load_ext tensorboard

In [14]:
%tensorboard --logdir "$tensorboard_dir"

## II - Partial fine-tuning: Use pretrained DenseNet and train specific layers with your data
You can set the second dataset with more images to run the second experiment that uses training steps. 

In this example, I run a second experiment with 1500 images (from both nodes).
The dataset is defined below, after TrainingPlan as previously shown.

You could also import the model you saved to perform your second TrainingPlan experiment (let's see below)



In [15]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

2024-02-23 10:03:58,042 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-23 10:03:58,044 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

{'config_mednist_1_sampled': [{'name': 'MedNIST_1_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [500, 3, 64, 64],
   'dataset_id': 'dataset_a6a459ef-91b9-45d0-8ba9-6ce940554dc8',
   'dataset_parameters': None}],
 'config_mednist_2_sampled': [{'name': 'MedNIST_2_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [1000, 3, 64, 64],
   'dataset_id': 'dataset_6f770f6b-3f0e-4aac-8cc9-60f43880dff5',
   'dataset_parameters': None}]}

Here I freeze 3 layers since we have a bigger dataset than in the first part

In [16]:
from fedbiomed.common.training_plans import TorchTrainingPlan
class MyTrainingPlan2(TorchTrainingPlan):

    def init_model(self, model_args):

        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=True)
       
        # For example, let's freeze layers of the last dense block
        for param in model.features[:-3].parameters():
            param.requires_grad = False

        # add the classifier 
        num_ftrs = model.classifier.in_features
        num_classes = model_args['num_classes'] 
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)       
            )
        
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    def training_data(self):
        
        # Custom torch Dataloader for MedNIST data and transform images and perform data augmentation 
       
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                #transforms.RandomHorizontalFlip(p=0.5),
                #transforms.RandomVerticalFlip(p=0.5),
                #transforms.RandomRotation(30),
                #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)



    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [17]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, # You could decrease the learning rate
    'epochs': 1, # you can increase the epoch's number =10
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}
model_args={
    'num_classes': 6
}
tags =  ['#MEDNIST', '#dataset']
rounds = 1  # you can increase the rounds's number 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan2,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1)
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)
    

2024-02-23 10:04:01,306 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-23 10:04:01,308 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 10:04:01,310 fedbiomed INFO - Node selected for training -> config_mednist_1_sampled

2024-02-23 10:04:01,310 fedbiomed INFO - Node selected for training -> config_mednist_2_sampled

2024-02-23 10:04:01,312 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/Documents/github/fedbiomed/var/experiments/Experiment_0001/model_75ee9d4f-9e1d-4232-91ae-4fdc7cfb4548.py

...

2024-02-23 10:04:01,407 fedbiomed INFO - Removing tensorboard logs from previous experiment

2024-02-23 10:04:01,410 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-23 10:04:01,411 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-23 10:04:01,412 fedbiomed DEBUG - Experimentation training_args updated for `job`

True

### II - 1. (Optional) Import a "custom model" or continue with the original DenseNet model of the TrainingPlan 

In [18]:
exp.training_plan().import_model('./training_plan1_densenet_MedNIST') 

2024-02-23 10:04:03,825 fedbiomed WARNING - 'TorchModel.set_weights' received inputs that did not cover alltrainable model parameters; missing weights: {'classifier.2.bias', 'classifier.2.weight'}

2024-02-23 10:04:03,826 fedbiomed WARNING - 'TorchModel.set_weights' received inputs with unexpected names: ['classifier.3.weight', 'classifier.3.bias']

### II - 2. Run your experiment 

In [19]:
exp.run()

2024-02-23 10:04:04,132 fedbiomed INFO - Sampled nodes in round 0 ['config_mednist_1_sampled', 'config_mednist_2_sampled']

2024-02-23 10:04:04,141 fedbiomed INFO - Sending request 
					 To: config_mednist_1_sampled 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-02-23 10:04:04,142 fedbiomed INFO - Sending request 
					 To: config_mednist_2_sampled 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-02-23 10:04:04,294 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 10:04:04,318 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

...

2024-02-23 10:13:48,885 fedbiomed INFO - Nodes that successfully reply in round 0 ['config_mednist_1_sampled', 'config_mednist_2_sampled']

2024-02-23 10:13:48,959 fedbiomed INFO - Saved aggregated params for round 0 in /home/ybouilla/Documents/github/fedbiomed/var/experiments/Experiment_0001/aggregated_params_698dd230-5aca-4c76-8003-7b8ededf7317.mpk

1

2024-02-23 10:53:12,727 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-23 11:02:49,072 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

For example,  At the end of training experiment, I obtained

                    fedbiomed INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 1.00000
					 ---------

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')


### II -  3. Export your model 

In [ ]:
#save model 
exp.training_plan().export_model('./training_plan2_densenet_MedNIST')

### II - 4. Display losses on Tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir "$tensorboard_dir"

### II - 5. Save and Import your model and parameters 

You could import your first model from TrainingPlan1 instead of loading the original DenseNet.
You could also retrieve the model's features.

In [ ]:
# save your model ( all layers model of te training experiment)
remote_model = exp.training_plan().model()  
torch.save(remote_model, './training_plan2_model')


In [ ]:
# import your model 
model= torch.load('./training_plan2_model')
model

### II - 6. Save model's features, parameters 

In [ ]:
model_features = exp.training_plan().export_model('./training_plan2_model')
model_features

In [ ]:
# import your model's layers features
model_features_= torch.load('./training_plan2_model')
model_features_

In [ ]:
# equivalent commands 
remote_model = exp.training_plan().model()
remote_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])